In [ ]:
!pip uninstall tensorflow-gpu -y
!pip install tensorflow-gpu==2.1.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import os
import pickle
from matplotlib import pyplot as plt
import cv2
import time
print(tf.__version__)

2.1.0


#Data Loader

In [ ]:
WORKERS = tf.data.experimental.AUTOTUNE

In [ ]:
class Dataset():
  def __init__(self, image_size, batch_size = 64):
    data_dir = "/content/drive/My Drive/Text to Image/dataset/birds"
    train_dir = data_dir + "/train"
    test_dir = data_dir + "/test"
    train_embeddings_path = train_dir + "/char-CNN-RNN-embeddings.pickle"
    test_embeddings_path = test_dir + "/char-CNN-RNN-embeddings.pickle"
    train_filenames_path = train_dir + "/filenames.pickle"
    test_filenames_path = test_dir + "/filenames.pickle"
    cub_dataset_dir = "/content/drive/My Drive/Text to Image/dataset/CUB_200_2011/CUB_200_2011"
    bounding_boxes_path = cub_dataset_dir + "/bounding_boxes.txt"
    image_ids_path = cub_dataset_dir + "/images.txt"
    images_path = cub_dataset_dir + "/images"
    
    self.image_width = image_size[0]
    self.image_height = image_size[1]
    self.batch_size = batch_size

    with open(train_filenames_path, 'rb') as f:
      self.train_filenames = pickle.load(f, encoding='latin1')
      self.train_filenames = [os.path.join(images_path, filename) + '.jpg' for filename in self.train_filenames]
    
    with open(test_filenames_path, 'rb') as f:
      self.test_filenames = pickle.load(f, encoding='latin1')
      self.test_filenames = [os.path.join(images_path, filename) + '.jpg' for filename in self.test_filenames]

    with open(train_embeddings_path, 'rb') as f:
      self.train_embeddings = pickle.load(f, encoding = 'latin1')

    with open(test_embeddings_path, 'rb') as f:
      self.test_embeddings = pickle.load(f, encoding = 'latin1')

    bounding_boxes = {}
    with open(bounding_boxes_path, 'rb') as f:
      box_coordinates = f.read()
      box_coordinates = box_coordinates.splitlines()
      box_coordinates = [box_coordinate.decode('utf-8') for box_coordinate in box_coordinates]
      for i in range(len(box_coordinates)):
        bounding_box = box_coordinates[i].split()
        bounding_boxes[bounding_box[0]] = [int(float(c)) for c in box_coordinates[i].split()][1:]

    image_ids_mapping = {}
    with open(image_ids_path, 'rb') as f:
      image_ids = f.read()
      image_ids = image_ids.splitlines()
      image_ids = [image_id.decode('utf-8') for image_id in image_ids]
      for i in range(len(image_ids)):
        image_id = image_ids[i].split()
        image_ids_mapping[image_id[0]] = image_id[1]

    bounding_boxes_mapping = {}
    for image_id in bounding_boxes.keys():
      bounding_boxes_mapping[images_path + "/" + image_ids_mapping[image_id]] = bounding_boxes[image_id]

    self.train_bounding_boxes = []
    self.test_bounding_boxes = []
    for i in range(len(self.train_filenames)):
      self.train_bounding_boxes.append(bounding_boxes_mapping[self.train_filenames[i]])
    for i in range(len(self.test_filenames)):
      self.test_bounding_boxes.append(bounding_boxes_mapping[self.test_filenames[i]])

  def crop(self, image, bounding_box):
    image = image.numpy()
    if bounding_box is not None:
      x, y, width, height = bounding_box
      image = image[y:(y + height), x:(x + width)]
      image = cv2.resize(image, (self.image_width, self.image_height))
    return image

  def parse_function(self, image_path, embeddings, bounding_box):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels = 3)
    # image = tf.image.crop_to_bounding_box(image, bounding_box[1], bounding_box[0], bounding_box[3], bounding_box[2])
    image = tf.py_function(func = self.crop, inp = [image, bounding_box], Tout = tf.float32)
    image.set_shape([self.image_width, self.image_height, 3])
    image = (image - 127.5) / 127.5
    # image = tf.image.resize(image, [self.image_width, self.image_height])

    embedding_index = np.random.randint(0, embeddings.shape[0] - 1)
    embedding = embeddings[embedding_index]
    return image, embedding
  
  def get_train_ds(self):
    BUFFER_SIZE = len(self.train_filenames)
    ds = tf.data.Dataset.from_tensor_slices((self.train_filenames, self.train_embeddings, self.train_bounding_boxes))
    ds = ds.shuffle(BUFFER_SIZE)
    ds = ds.repeat()
    ds = ds.map(self.parse_function, num_parallel_calls = WORKERS)
    ds = ds.batch(self.batch_size, drop_remainder = True)
    ds = ds.prefetch(5)
    return ds
  
  def get_test_ds(self):
    BUFFER_SIZE = len(self.test_filenames)
    ds = tf.data.Dataset.from_tensor_slices((self.test_filenames, self.test_embeddings, self.test_bounding_boxes))
    ds = ds.shuffle(BUFFER_SIZE)
    ds = ds.repeat(1)
    ds = ds.map(self.parse_function, num_parallel_calls = WORKERS)
    ds = ds.batch(self.batch_size, drop_remainder = True)
    ds = ds.prefetch(1)
    return ds

#Stage 1 StackGAN

In [ ]:
"""class generate_condition(layers.layer):
  def __init__(self):
    super(generate_condition, self).__init__()
    self.dense = tf.keras.layers.Dense(units = 256)


  def call(self, c_var):
    X=tf.keras.layers.Flatten(c_var)
    X = self.dense(X)
    phi = tf.nn.leaky_relu(X)
    mean = phi[:, :128]
    std = phi[:, 128:]
    epsilon = K.random_normal(shape = K.constant((mean.shape[1], ), dtype = 'int32'))
    C = mean + epsilon*std
    return mean, std"""

"class generate_condition(layers.layer):\n  def __init__(self):\n    super(generate_condition, self).__init__()\n    self.dense = tf.keras.layers.Dense(units = 256)\n\n\n  def call(self, c_var):\n    X=tf.keras.layers.Flatten(c_var)\n    X = self.dense(X)\n    phi = tf.nn.leaky_relu(X)\n    mean = phi[:, :128]\n    std = phi[:, 128:]\n    epsilon = K.random_normal(shape = K.constant((mean.shape[1], ), dtype = 'int32'))\n    C = mean + epsilon*std\n    return mean, std"

In [ ]:

"""class sample_encoded_context(tf.keras.Model):
  def __init__(self):
    super(ConditioningAugmentation, self).__init__()
    self.dense = tf.keras.layers.Dense(units = 256)

  def call(self, E):
    X = self.dense(E)
    phi = tf.nn.leaky_relu(X)
    mean = phi[:, :128]
    std = K.exp(phi[:, 128:])
    epsilon = K.random_normal(shape = K.constant((mean.shape[1], ), dtype = 'int32'))
    C = mean + epsilon*std
    return C, phi"""

"class sample_encoded_context(tf.keras.Model):\n  def __init__(self):\n    super(ConditioningAugmentation, self).__init__()\n    self.dense = tf.keras.layers.Dense(units = 256)\n\n  def call(self, E):\n    X = self.dense(E)\n    phi = tf.nn.leaky_relu(X)\n    mean = phi[:, :128]\n    std = K.exp(phi[:, 128:])\n    epsilon = K.random_normal(shape = K.constant((mean.shape[1], ), dtype = 'int32'))\n    C = mean + epsilon*std\n    return C, phi"

In [ ]:
"""def sample_encoded_context(self, embeddings):
        '''Helper function for init_opt'''
        c_mean_logsigma = self.model.generate_condition(embeddings)
        mean = c_mean_logsigma[0]
        if cfg.TRAIN.COND_AUGMENTATION:
            # epsilon = tf.random_normal(tf.shape(mean))
            epsilon = tf.truncated_normal(tf.shape(mean))
            stddev = tf.exp(c_mean_logsigma[1])
            c = mean + stddev * epsilon

            kl_loss = KL_loss(c_mean_logsigma[0], c_mean_logsigma[1])
        else:
            c = mean
            kl_loss = 0

        return c, cfg.TRAIN.COEFF.KL * kl_loss"""

"def sample_encoded_context(self, embeddings):\n        '''Helper function for init_opt'''\n        c_mean_logsigma = self.model.generate_condition(embeddings)\n        mean = c_mean_logsigma[0]\n        if cfg.TRAIN.COND_AUGMENTATION:\n            # epsilon = tf.random_normal(tf.shape(mean))\n            epsilon = tf.truncated_normal(tf.shape(mean))\n            stddev = tf.exp(c_mean_logsigma[1])\n            c = mean + stddev * epsilon\n\n            kl_loss = KL_loss(c_mean_logsigma[0], c_mean_logsigma[1])\n        else:\n            c = mean\n            kl_loss = 0\n\n        return c, cfg.TRAIN.COEFF.KL * kl_loss"

In [ ]:
def KL_loss(y_true, y_pred):
  mean = y_pred[:, :128]
  logsigma = y_pred[:, 128:]
  loss = -logsigma + 0.5*(-1 + K.exp(2.0*logsigma) + K.square(mean))
  loss = K.mean(loss)
  return loss

In [ ]:
class ConditioningAugmentation(tf.keras.Model):
  def __init__(self):
    super(ConditioningAugmentation, self).__init__()
    self.dense = tf.keras.layers.Dense(units = 256)

  def call(self, E):
    X=tf.keras.layers.Flatten()(E)
    X = self.dense(X)
    phi = tf.nn.leaky_relu(X)
    mean = phi[:, :128]
    std = K.exp(phi[:, 128:])
    #epsilon = K.random_normal(shape = K.constant((mean.shape[1], ), dtype = 'int32'))
    epsilon=tf.keras.backend.truncated_normal(shape=(mean.shape))
    C = mean + epsilon*std
    return C, phi

In [ ]:
class EmbeddingCompressor(tf.keras.Model):
  def __init__(self):
    super(EmbeddingCompressor, self).__init__()
    self.dense = tf.keras.layers.Dense(units = 128)

  def call(self, E):
    X = self.dense(E)
    return tf.nn.relu(X)

In [ ]:
class Stage1Generator(tf.keras.Model):
  def __init__(self,image_shape=[64,64]):
    super(Stage1Generator, self).__init__()
    self.s = image_shape[0]
    self.s2, self.s4, self.s8, self.s16 = int(self.s / 2), int(self.s / 4), int(self.s / 8), int(self.s / 16)
    self.canet = ConditioningAugmentation()
    self.concat = tf.keras.layers.Concatenate(axis = 1)
    self.dense = tf.keras.layers.Dense(units =128*4*4*8 , kernel_initializer = tf.random_normal_initializer(stddev = 0.02))
    self.reshape = tf.keras.layers.Reshape(target_shape = (4, 4, 128*8), input_shape = (128*8*4*4, ))
    self.batchnorm1 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.9)
    
    #self.resblock1= Resblock([256,256,1024],[1,3,3])
    self.conv5 = tf.keras.layers.Conv2D(256,1,strides=1,padding="same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm5= tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv6= tf.keras.layers.Conv2D(256,3,strides=1,padding="same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm6 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)

    self.conv7= tf.keras.layers.Conv2D(1024,3,strides=1,padding="same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm7 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)

    self.conv8=tf.keras.layers.Conv2D(512,3,1,padding='same', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm8 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)

    #self.resblock2=Resblock([256,256,512],[1,3,3])
    self.conv9 = tf.keras.layers.Conv2D(128,1,strides=1,padding="same",kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm9= tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv10= tf.keras.layers.Conv2D(128,3,strides=1,padding="same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm10 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv11= tf.keras.layers.Conv2D(512,3,strides=1,padding="same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm11 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
  
    self.conv3=tf.keras.layers.Conv2D(256,3,1,padding='same', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm3 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv4=tf.keras.layers.Conv2D(128,3,1,padding='same', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm4 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
  
    self.conv12=tf.keras.layers.Conv2D(3,3,1,padding='same', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    

  def call(self, inputs,istraining=True):
    E, Z = inputs
    C, phi = self.canet(E)

    gen_input = self.concat([C, Z])
    gen_input = tf.keras.layers.Flatten()(gen_input)
    X = self.dense(gen_input)
    X = self.batchnorm1(X) ##
    R = self.reshape(X) ##
    ## X = self.batchnorm1(X)
    ## R = tf.nn.relu(X)
    
    
    #X = tf.nn.relu(X

    #X = self.resblock1(X)
    X=self.conv5(R)
    X=self.batchnorm5(X,training=istraining)
    X=tf.nn.relu(X)

    X=self.conv6(X)
    X=self.batchnorm6(X,training=istraining)
    X=tf.nn.relu(X)

    X=self.conv7(X)
    X=self.batchnorm7(X,training=istraining)

    X = tf.math.add(R,X)
    X=tf.nn.relu(X)

    X = tf.compat.v1.image.resize_nearest_neighbor(X, [self.s8, self.s8], align_corners=False, name=None, half_pixel_centers=False)
    X= self.conv8(X)
    R2= self.batchnorm8(X,training=istraining)

    #X= self.resblock2(X)
    X=self.conv9(R2)
    X=self.batchnorm9(X,training=istraining)
    X=tf.nn.relu(X)

    X=self.conv10(X)
    X=self.batchnorm10(X,training=istraining)
    X=tf.nn.relu(X)

    X=self.conv11(X)
    X=self.batchnorm11(X,training=istraining)
    
    X = tf.math.add(X,R2)
    X=tf.nn.relu(X)

    X = tf.compat.v1.image.resize_nearest_neighbor(X, [self.s4, self.s4], align_corners=False, name=None, half_pixel_centers=False)
    X = self.conv3(X)
    X= self.batchnorm3(X,training=istraining)
    X=tf.nn.relu(X)
    X = tf.compat.v1.image.resize_nearest_neighbor(X, [self.s2, self.s2], align_corners=False, name=None, half_pixel_centers=False)
    X = self.conv4(X)
    X= self.batchnorm4(X,training=istraining)
    X=tf.nn.relu(X)
    X = tf.compat.v1.image.resize_nearest_neighbor(X, [self.s, self.s], align_corners=False, name=None, half_pixel_centers=False)
    X = self.conv12(X)


    
    return tf.nn.tanh(X), phi

In [ ]:
class Stage1Discriminator(tf.keras.Model):
  def __init__(self):
    super(Stage1Discriminator, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.conv2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm1 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv3 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm2 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv4 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm3 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv5 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 1, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm4 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv6 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm5 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv7 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm6 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.embed = EmbeddingCompressor()
    self.reshape = tf.keras.layers.Reshape(target_shape = (1, 1, 128))
    self.concat = tf.keras.layers.Concatenate()
    self.conv8 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 1, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm7 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.keras.initializers.RandomNormal(1.0,0.02),axis = -1, momentum = 0.9)
    self.conv9 = tf.keras.layers.Conv2D(filters = 1, kernel_size = 4, strides = 4, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))

  def call(self, inputs):
    I, E = inputs
    X = self.conv1(I)
    X = tf.nn.leaky_relu(X)

    X = self.conv2(X)
    X = self.batchnorm1(X)
    X = tf.nn.leaky_relu(X)

    X = self.conv3(X)
    X = self.batchnorm2(X)


    X = self.conv4(X)
    R = self.batchnorm3(X)
    

    X = self.conv5(R)
    X = self.batchnorm4(X)
    X = tf.nn.leaky_relu(X)

    X = self.conv6(X)
    X = self.batchnorm5(X)
    X = tf.nn.leaky_relu(X)

    X = self.conv7(X)
    X = self.batchnorm6(X)
    X = tf.math.add(X,R)
    X = tf.nn.leaky_relu(X)

    T = self.embed(E)
    T = self.reshape(T)
    T = tf.tile(T, (1, 4, 4, 1))
    merged_input = self.concat([X, T])

    Y = self.conv8(merged_input)
    Y = self.batchnorm7(Y)
    Y = tf.nn.leaky_relu(Y)

    Y = self.conv9(Y)
    return tf.squeeze(Y)

In [ ]:
class Stage1Model(tf.keras.Model):
  def __init__(self):
    super(Stage1Model, self).__init__()
    self.stage1_generator = Stage1Generator()
    self.stage1_discriminator = Stage1Discriminator()

  def train(self, train_ds, batch_size = 64, num_epochs = 600, z_dim = 100, c_dim = 128, stage1_generator_lr = 0.0002, stage1_discriminator_lr = 0.0002):
    generator_optimizer = tf.keras.optimizers.Adam(lr = stage1_generator_lr, beta_1 = 0.5, beta_2 = 0.999)
    discriminator_optimizer = tf.keras.optimizers.Adam(lr = stage1_discriminator_lr, beta_1 = 0.5, beta_2 = 0.999)
    
    for epoch in range(num_epochs):
      print("Epoch %d/%d:\n ["%(epoch + 1, num_epochs), end = "")
      start_time = time.time()
      if epoch % 50 == 0:
        K.set_value(generator_optimizer.learning_rate, generator_optimizer.learning_rate / 2)
        K.set_value(discriminator_optimizer.learning_rate, discriminator_optimizer.learning_rate / 2)
    
      generator_loss_log = []
      discriminator_loss_log = []
      steps_per_epoch = 125
      batch_iter = iter(train_ds)
      for i in range(steps_per_epoch):
        if i % 5 == 0:
          print("=", end = "")
        image_batch, embedding_batch = next(batch_iter)
        z_noise = tf.random.normal((batch_size, z_dim))

        mismatched_images = tf.roll(image_batch, shift = 1, axis = 0)

        real_labels = tf.random.uniform(shape = (batch_size, ), minval = 0.9, maxval = 1.0)
        fake_labels = tf.zeros(shape = (batch_size, ))
        mismatched_labels = tf.zeros(shape = (batch_size, ))
        disc_labels=tf.concat([real_labels,fake_labels],axis=0)

        with tf.GradientTape() as generator_tape, tf.GradientTape() as discriminator_tape:
          fake_images, phi = self.stage1_generator([embedding_batch, z_noise])
          
          
          real_logits = self.stage1_discriminator([image_batch, embedding_batch])
          fake_logits =  self.stage1_discriminator([fake_images, embedding_batch])
          mismatched_logits = self.stage1_discriminator([mismatched_images, embedding_batch])
          disc_logits=tf.concat([real_logits,fake_logits],axis=0)
         
          l_sup = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(real_labels, fake_logits))
          l_klreg = KL_loss(tf.random.normal((phi.shape[0], phi.shape[1])), phi)
          generator_loss = l_sup + 2*l_klreg
          
          l_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(real_labels, real_logits))
          l_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(fake_labels, fake_logits))
          l_mismatched = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(mismatched_labels, mismatched_logits))
          discriminator_loss = tf.add(l_real,0.5*tf.add(l_mismatched,l_fake))
        
        generator_gradients = generator_tape.gradient(generator_loss, self.stage1_generator.trainable_variables)
        discriminator_gradients = discriminator_tape.gradient(discriminator_loss, self.stage1_discriminator.trainable_variables)
        
        generator_optimizer.apply_gradients(zip(generator_gradients, self.stage1_generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.stage1_discriminator.trainable_variables))
        
        generator_loss_log.append(generator_loss)
        discriminator_loss_log.append(discriminator_loss)

      end_time = time.time()

      if epoch % 1 == 0:
        epoch_time = end_time - start_time
        template = "] - generator_loss: {:.4f} - discriminator_loss: {:.4f} - epoch_time: {:.2f} s"
        print(template.format(tf.reduce_mean(generator_loss_log), tf.reduce_mean(discriminator_loss_log), epoch_time))

      if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
        save_path = "/content/drive/My Drive/Text to Image/lr_results/epoch_" + str(epoch + 1)
        temp_embeddings = None
        for _, embeddings in train_ds:
          temp_embeddings = embeddings.numpy()
          break
        if os.path.exists(save_path) == False:
          os.makedirs(save_path)
        temp_batch_size = 10
        temp_z_noise = tf.random.normal((temp_batch_size, z_dim))
        temp_embedding_batch = temp_embeddings[0:temp_batch_size]
        fake_images, _ = self.stage1_generator([temp_embedding_batch, temp_z_noise],istraining=False)
        for i, image in enumerate(fake_images):
          image = 127.5*image + 127.5
          image = image.numpy().astype('uint8')
          cv2.imwrite(save_path + "/gen_%d.png"%(i), image)
      
        self.stage1_generator.save_weights("/content/drive/My Drive/Text to Image/lr_model_checkpoints/stage1_generator_" + str(epoch + 1) + ".ckpt")
        self.stage1_discriminator.save_weights("/content/drive/My Drive/Text to Image/lr_model_checkpoints/stage1_discriminator_" + str(epoch + 1) + ".ckpt")

    def generate_image(self, embedding):
      self.stage1_generator.compile(loss = "mse", optimizer = "adam")
      self.stage1_generator.load_weights("/content/drive/My Drive/Text to Image/lr_model_checkpoints/stage1_generator_600.ckpt").expect_partial()
      z_noise = tf.random.normal((batch_size, z_dim))
      generated_image = self.stage1_generator([embedding, z_noise])
      return generated_image

In [ ]:
dataset = Dataset(image_size = (256, 256),batch_size=16)
train_ds = dataset.get_train_ds()

In [ ]:
model = Stage1Model()
model.train(train_ds)

Epoch 1/600:
 [=========================] - generator_loss: 0.7553 - discriminator_loss: 1.3827 - epoch_time: 47.31 s
Epoch 2/600:
 [=========================] - generator_loss: 0.7576 - discriminator_loss: 1.3787 - epoch_time: 50.21 s
Epoch 3/600:
 [=========================] - generator_loss: 0.8260 - discriminator_loss: 1.3554 - epoch_time: 51.85 s
Epoch 4/600:
 [=========================] - generator_loss: 0.7702 - discriminator_loss: 1.3738 - epoch_time: 53.48 s
Epoch 5/600:
 [=========================] - generator_loss: 0.8534 - discriminator_loss: 1.3456 - epoch_time: 52.69 s
Epoch 6/600:
 [=========================] - generator_loss: 0.8513 - discriminator_loss: 1.3476 - epoch_time: 52.75 s
Epoch 7/600:
 [=========================] - generator_loss: 0.8720 - discriminator_loss: 1.3339 - epoch_time: 49.13 s
Epoch 8/600:
 [=========================] - generator_loss: 0.9012 - discriminator_loss: 1.3109 - epoch_time: 53.52 s
Epoch 9/600:
 [=========================] - generator_lo

#Stage 2 StackGAN

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
  def __init__(self):
    super(ResidualBlock, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters = 128*4, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm1 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv2 = tf.keras.layers.Conv2D(filters = 128*4, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm2 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)

    def call(self, I):
      X = self.conv1(I)
      X = self.batchnorm1(X)
      X = tf.nn.relu(X)

      X = self.conv2(X)
      X = self.batchnorm2(X)
      X = tf.nn.relu(X)
      X = tf.keras.layers.Add()([X, I])
      X = tf.nn.relu(X)
      return X

In [ ]:

class Stage2Generator(tf.keras.Model):
  def __init__(self):
    super(Stage2Generator, self).__init__()
    self.canet = ConditioningAugmentation()
    self.conv1 = tf.keras.layers.Conv2D(128, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.conv2 = tf.keras.layers.Conv2D(256, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm1 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv3 = tf.keras.layers.Conv2D(512, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm2 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv4 = tf.keras.layers.Conv2D(512, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm3 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.resblock1 = ResidualBlock()
    self.resblock2 = ResidualBlock()
    self.resblock3 = ResidualBlock()
    self.resblock4 = ResidualBlock()
    self.upsamp1 = tf.keras.layers.UpSampling2D(size = (2, 2))
    self.conv5 = tf.keras.layers.Conv2D(256, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm4 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.upsamp2 = tf.keras.layers.UpSampling2D(size = (2, 2))
    self.conv6 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm5 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.upsamp3 = tf.keras.layers.UpSampling2D(size = (2, 2))
    self.conv7 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm6 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.upsamp4 = tf.keras.layers.UpSampling2D(size = (2, 2))
    self.conv8 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm7 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv9 = tf.keras.layers.Conv2D(filters = 3, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
  
  def call(self, inputs):
    E, I = inputs
    C, phi = self.canet(E)

    X = self.conv1(I)
    X = tf.nn.relu(X)
    
    X = self.conv2(X)
    X = self.batchnorm1(X)
    X = tf.nn.relu(X)

    X = self.conv3(X)
    X = self.batchnorm2(X)
    X = tf.nn.relu(X)

    C = K.expand_dims(C, axis = 1)
    C = K.expand_dims(C, axis = 1)
    C = K.tile(C, [1, 16, 16, 1])
    J = K.concatenate([C, X], axis = 3)

    X = self.conv4(X)
    X = self.batchnorm3(X)
    X = tf.nn.relu(X)

    X = self.resblock1(X)
    X = self.resblock2(X)
    X = self.resblock3(X)
    X = self.resblock4(X)

    X = self.upsamp1(X)
    X = self.conv5(X)
    X = self.batchnorm4(X)
    X = tf.nn.relu(X)
    
    X = self.upsamp2(X)
    X = self.conv6(X)
    X = self.batchnorm5(X)
    X = tf.nn.relu(X)
    
    X = self.upsamp3(X)
    X = self.conv7(X)
    X = self.batchnorm6(X)
    X = tf.nn.relu(X)
    
    X = self.upsamp4(X)
    X = self.conv8(X)
    X = self.batchnorm7(X)
    X = tf.nn.relu(X)
    
    X = self.conv9(X)
    return tf.nn.tanh(X), phi

In [ ]:
class Stage2Discriminator(tf.keras.Model):
  def __init__(self):
    super(Stage2Discriminator, self).__init__()
    self.embed = EmbeddingCompressor()
    self.reshape = tf.keras.layers.Reshape(target_shape = (1, 1, 128))
    self.conv1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.conv2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm1 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv3 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm2 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv4 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm3 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv5 = tf.keras.layers.Conv2D(filters = 1024, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm4 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv6 = tf.keras.layers.Conv2D(filters = 2048, kernel_size = 4, strides = 2, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm5 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv7 = tf.keras.layers.Conv2D(filters = 1024, kernel_size = 1, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm6 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv8 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 1, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm7 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv9 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 1, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm8 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv10 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm9 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv11 = tf.keras.layers.Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm10 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)

    self.conv12 = tf.keras.layers.Conv2D(filters = 64*8, kernel_size = 1, strides = 1, padding = "same", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))
    self.batchnorm11 = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)
    self.conv13 = tf.keras.layers.Conv2D(filters = 1, kernel_size = 4, strides = 4, padding = "valid", kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = 0.02))

  def call(self, inputs):
    I, E = inputs
    T = self.embed(E)
    T = self.reshape(T)
    T = tf.tile(T, (1, 4, 4, 1))

    X = self.conv1(I)
    X = tf.nn.leaky_relu(X)

    X = self.conv2(X)
    X = self.batchnorm1(X)
    X = tf.nn.leaky_relu(X)
    
    X = self.conv3(X)
    X = self.batchnorm2(X)
    X = tf.nn.leaky_relu(X)
    
    X = self.conv4(X)
    X = self.batchnorm3(X)
    X = tf.nn.leaky_relu(X)
    
    X = self.conv5(X)
    X = self.batchnorm4(X)
    X = tf.nn.leaky_relu(X)
   
    X = self.conv6(X)
    X = self.batchnorm5(X)
    X = tf.nn.leaky_relu(X)
    
    X = self.conv7(X)
    X = self.batchnorm6(X)
    X = tf.nn.leaky_relu(X)
    
    X = self.conv8(X)
    X = self.batchnorm7(X)

    Y = self.conv9(X)
    Y = self.batchnorm8(Y)
    Y = tf.nn.leaky_relu(Y)

    Y = self.conv10(Y)
    Y = self.batchnorm9(Y)
    Y = tf.nn.leaky_relu(Y)

    Y = self.conv11(Y)
    Y = self.batchnorm10(Y)

    A = tf.keras.layers.Add()([X, Y])
    A = tf.nn.leaky_relu(A)

    merged_input = tf.keras.layers.concatenate([A, T])

    Z = self.conv12(merged_input)
    Z = self.batchnorm11(Z)
    Z = tf.nn.leaky_relu(Z)
    
    Z = self.conv13(Z)
    return tf.squeeze(Z)

In [ ]:
class Stage2Model(tf.keras.Model):
  def __init__(self):
    super(Stage2Model, self).__init__()
    self.stage1_generator = Stage1Generator()
    self.stage1_generator.compile(loss = "mse", optimizer = "adam")
    self.stage1_generator.load_weights("/content/drive/My Drive/Text to Image/lr_model_checkpoints/stage1_generator_600.ckpt").expect_partial()
    self.stage2_generator = Stage2Generator()
    self.stage2_discriminator = Stage2Discriminator()

   
    
  ### lr
  def train(self, train_ds, batch_size = 16, num_epochs = 1200, z_dim = 100, stage1_generator_lr = 0.0002, stage1_discriminator_lr = 0.0002):
    generator_optimizer = tf.keras.optimizers.Adam(lr = stage1_generator_lr, beta_1 = 0.5, beta_2 = 0.999)
    discriminator_optimizer = tf.keras.optimizers.Adam(lr = stage1_discriminator_lr, beta_1 = 0.5, beta_2 = 0.999)
    
    for epoch in range(0, num_epochs): ### 170
      print("Epoch %d/%d:\n ["%(epoch + 1, num_epochs), end = "")
      start_time = time.time()
      if epoch % 100 == 0:
        K.set_value(generator_optimizer.learning_rate, generator_optimizer.learning_rate / 2)
        K.set_value(discriminator_optimizer.learning_rate, discriminator_optimizer.learning_rate / 2)
    
      generator_loss_log = []
      discriminator_loss_log = []
      steps_per_epoch = 125
      batch_iter = iter(train_ds)
      for i in range(steps_per_epoch):
        if i % 5 == 0:
          print("=", end = "")
        hr_image_batch, embedding_batch = next(batch_iter)
        z_noise = tf.random.normal((batch_size, z_dim))

        mismatched_images = tf.roll(hr_image_batch, shift = 1, axis = 0)

        real_labels = tf.random.uniform(shape = (batch_size, ), minval = 0.9, maxval = 1.0)
        fake_labels = tf.random.uniform(shape = (batch_size, ), minval = 0.0, maxval = 0.1)
        mismatched_labels = tf.random.uniform(shape = (batch_size, ), minval = 0.0, maxval = 0.1)

        with tf.GradientTape() as generator_tape, tf.GradientTape() as discriminator_tape:
          lr_fake_images, _ = self.stage1_generator([embedding_batch, z_noise])
          hr_fake_images, phi = self.stage2_generator([embedding_batch, lr_fake_images])
          real_logits = self.stage2_discriminator([hr_image_batch, embedding_batch])
          fake_logits = self.stage2_discriminator([hr_fake_images, embedding_batch])
          mismatched_logits = self.stage2_discriminator([mismatched_images, embedding_batch])
          
          l_sup = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(real_labels, fake_logits))
          l_klreg = KL_loss(tf.random.normal((phi.shape[0], phi.shape[1])), phi)
          generator_loss = l_sup + 2.0*l_klreg
          
          l_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(real_labels, real_logits))
          l_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(fake_labels, fake_logits))
          l_mismatched = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(mismatched_labels, mismatched_logits))
          discriminator_loss = 0.5*tf.add(l_real, 0.5*tf.add(l_fake, l_mismatched))
        
        generator_gradients = generator_tape.gradient(generator_loss, self.stage2_generator.trainable_variables)
        discriminator_gradients = discriminator_tape.gradient(discriminator_loss, self.stage2_discriminator.trainable_variables)
        
        generator_optimizer.apply_gradients(zip(generator_gradients, self.stage2_generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.stage2_discriminator.trainable_variables))
        
        generator_loss_log.append(generator_loss)
        discriminator_loss_log.append(discriminator_loss)
        
      end_time = time.time()

      if epoch % 1 == 0:
        epoch_time = end_time - start_time
        template = "] - generator_loss: {:.4f} - discriminator_loss: {:.4f} - epoch_time: {:.2f} s"
        print(template.format(tf.reduce_mean(generator_loss_log), tf.reduce_mean(discriminator_loss_log), epoch_time))

      if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
        save_path = "/content/drive/My Drive/Text to Image/hr_results/epoch_" + str(epoch + 1)
        temp_embeddings = None
        for _, embeddings in train_ds:
          temp_embeddings = embeddings.numpy()
          break
        if os.path.exists(save_path) == False:
          os.makedirs(save_path)
        temp_batch_size = 10
        temp_z_noise = tf.random.normal((temp_batch_size, z_dim))
        temp_embedding_batch = temp_embeddings[0:temp_batch_size]
        fake_images, _ = self.stage1_generator([temp_embedding_batch, temp_z_noise])
        for i, image in enumerate(fake_images):
          image = 127.5*image + 127.5
          image = image.numpy().astype('uint8')
          cv2.imwrite(save_path + "/gen_%d.png"%(i), image)
        self.stage2_generator.save_weights("/content/drive/My Drive/Text to Image/hr_model_checkpoints/stage2_generator_" + str(epoch + 1) + ".ckpt")
        self.stage2_discriminator.save_weights("/content/drive/My Drive/Text to Image/hr_model_checkpoints/stage2_discriminator_" + str(epoch + 1) + ".ckpt")

In [ ]:
dataset = Dataset(image_size = (256, 256))
train_ds = dataset.get_train_ds()

In [ ]:
stage2_model = Stage2Model()
stage2_model.train(train_ds)

Epoch 1/1200:
 [=========================] - generator_loss: 1.7323 - discriminator_loss: 0.6168 - epoch_time: 877.81 s
Epoch 2/1200:
 [=========================] - generator_loss: 1.4515 - discriminator_loss: 0.6344 - epoch_time: 571.64 s
Epoch 3/1200:
 [=========================] - generator_loss: 1.5055 - discriminator_loss: 0.6228 - epoch_time: 434.90 s
Epoch 4/1200:
 [=========================] - generator_loss: 2.1972 - discriminator_loss: 0.5870 - epoch_time: 344.37 s
Epoch 5/1200:
 [=========================] - generator_loss: 1.8523 - discriminator_loss: 0.6191 - epoch_time: 254.10 s
Epoch 6/1200:
 [=========================] - generator_loss: 2.0137 - discriminator_loss: 0.5890 - epoch_time: 212.04 s
Epoch 7/1200:
 [=========================] - generator_loss: 2.7649 - discriminator_loss: 0.5414 - epoch_time: 165.27 s
Epoch 8/1200:
 [=========================] - generator_loss: 2.8453 - discriminator_loss: 0.5315 - epoch_time: 137.99 s
Epoch 9/1200:
 [========================

In [ ]:
class Stage2Model(tf.keras.Model):
  def __init__(self):
    super(Stage2Model, self).__init__()
    self.stage1_generator = Stage1Generator()
    # self.stage1_generator.compile(loss = 'mse', optimizer = "adam")
    # self.stage1_generator.train_on_batch([K.ones((1, 1024)), K.ones((1, 100))], [K.ones((1, 64, 64, 3)), K.ones((1, 256))])
    self.stage1_generator.load_weights("/content/drive/My Drive/Text to Image/lr_model_checkpoints/stage1_generator_600.ckpt")
    self.stage2_generator = Stage2Generator()
    self.stage2_discriminator = Stage2Discriminator()

  def train(self, X_train, batch_size = 64, num_epochs = 1200, z_dim = 100, stage1_generator_lr = 0.0004, stage1_discriminator_lr = 0.0004):
    generator_optimizer = tf.keras.optimizers.Adam(lr = stage1_generator_lr, beta_1 = 0.5, beta_2 = 0.999)
    discriminator_optimizer = tf.keras.optimizers.Adam(lr = stage1_discriminator_lr, beta_1 = 0.5, beta_2 = 0.999)
    
    for epoch in range(num_epochs):
      print("Epoch %d/%d:\n ["%(epoch + 1, num_epochs), end = "")
      start_time = time.time()
      if epoch % 50 == 0:
        K.set_value(generator_optimizer.learning_rate, generator_optimizer.learning_rate / 2)
        K.set_value(discriminator_optimizer.learning_rate, discriminator_optimizer.learning_rate / 2)
    
      generator_loss_log = []
      discriminator_loss_log = []
      steps_per_epoch = 125
      batch_iter = iter(train_ds)
      for i in range(steps_per_epoch):
        if i % 5 == 0:
          print("=", end = "")
        hr_image_batch, embedding_batch = next(batch_iter)
        z_noise = tf.random.normal((batch_size, z_dim))

        mismatched_images = tf.roll(image_batch, shift = 1, axis = 0)

        real_labels = tf.random.uniform(shape = (batch_size, ), minval = 0.9, maxval = 1.0)
        fake_labels = tf.zeros(shape = (batch_size, ))
        mismatched_labels = tf.zeros(shape = (batch_size, ))
        disc_labels=tf.concat([real_labels,fake_labels],axis=0)

        with tf.GradientTape() as generator_tape, tf.GradientTape() as discriminator_tape:
          lr_fake_images, _ = self.stage1_generator([embedding_batch, z_noise])
          hr_fake_images, phi = self.stage2_generator([embedding_batch, lr_fake_images])
          real_logits = self.stage2_discriminator([hr_image_batch, embedding_batch])
          fake_logits = self.stage2_discriminator([hr_fake_images, embedding_batch])
          mismatched_logits = self.stage2_discriminator([mismatched_images, embedding_batch])
          
          l_sup = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(real_labels, fake_logits))
          l_klreg = KL_loss(tf.random.normal((phi.shape[0], phi.shape[1])), phi)
          generator_loss = l_sup + 2*l_klreg
          
          l_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(real_labels, real_logits))
          l_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(fake_labels, fake_logits))
          l_mismatched = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(mismatched_labels, mismatched_logits))
          discriminator_loss = tf.add(l_real, 0.5*tf.add(l_fake, l_mismatched))
        
        generator_gradients = generator_tape.gradient(generator_loss, self.stage2_generator.trainable_variables)
        discriminator_gradients = discriminator_tape.gradient(discriminator_loss, self.stage2_discriminator.trainable_variables)
        
        generator_optimizer.apply_gradients(zip(generator_gradients, self.stage2_generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.stage2_discriminator.trainable_variables))
        
        generator_loss_log.append(generator_loss)
        discriminator_loss_log.append(discriminator_loss)
        
      end_time = time.time()

      if epoch % 1 == 0:
        epoch_time = end_time - start_time
        template = "Epoch {:03d}: generator_loss: {:.4f} - discriminator_loss: {:.4f} - epoch_time: {:.2f} s"
        print(template.format(epoch + 1, tf.reduce_mean(generator_loss_log), tf.reduce_mean(discriminator_loss_log), epoch_time))

      if epoch % 50 == 0 or epoch == num_epochs - 1:
        save_path = "/content/drive/My Drive/Text to Image/hr_results/epoch_" + str(epoch + 1)
        if os.path.exists(save_path) == False:
          os.makedirs(save_path)
        temp_batch_size = 10
        temp_z_noise = tf.random.normal((temp_batch_size, z_dim))
        temp_embedding_batch = train_embeddings[0:temp_batch_size]
        fake_images, _ = self.stage1_generator([temp_embedding_batch, temp_z_noise])
        for i, image in enumerate(fake_images):
          image = 127.5*image + 127.5
          image = image.numpy().astype('uint8')
          cv2.imwrite(save_path + "/gen_%d.png"%(i), image)
        self.stage2_generator.save_weights("/content/drive/My Drive/Text to Image/stage2_model_weights/stage1_generator_" + str(epoch + 1) + ".ckpt")
        self.stage2_discriminator.save_weights("/content/drive/My Drive/Text to Image/stage2_model_weights/stage1_discriminator_" + str(epoch + 1) + ".ckpt")
   
    def generate_image(self, embedding):
      self.stage2_generator.compile(loss = "mse", optimizer = "adam")
      self.stage2_generator.load_weights("/content/drive/My Drive/Text to Image/lr_model_checkpoints/stage1_generator_600.ckpt").expect_partial()
      z_noise = tf.random.normal((batch_size, z_dim))
      generated_image = self.stage1_generator([embedding, z_noise])
     
      return generated_image


class ResBlock(tf.keras.Model):
  def __init__(self,filters,filters_shapes,pads="same",strides=1,use_bias=True):#(3,3)
    super(ResBlock,self).__init__()
    filter1, filter2, filter3 = filters
    filter1_shape, filter2_shape, filter3_shape = filters_shapes

    self.conv1 = tf.keras.layers.Conv2D(filter1, filter1_shape,strides=strides,padding="same",use_bias=use_bias)
    self.batchNorm1= tf.keras.layers.BatchNormalization()

    self.conv2= tf.keras.layers.Conv2D(filter2, filter2_shape,strides=strides,padding="same",use_bias=use_bias)
    self.batchNorm2 = tf.keras.layers.BatchNormalization()

    self.conv3= tf.keras.layers.Conv2D(filter3, filter3_shape,strides=strides,padding="same",use_bias=use_bias)
    self.batchNorm3 = tf.keras.layers.BatchNormalization()

  def call(self,input_tensor,is_training=True):
    x=self.conv1(input_tensor)
    x=self.batchNorm1(x,training=is_training)
    x=tf.nn.relu(x)

    x=self.conv2(x)
    x=self.batchNorm2(x,training=is_training)
    x=tf.nn.relu(x)

    x=self.conv3(x)
    x=self.batchNorm3(x,training=is_training)
    

    x = tf.add(x,input_tensor)
    return tf.nn.relu(x) 



In [ ]:
class Stage1Model(tf.keras.Model):
  def __init__(self):
    super(Stage1Model, self).__init__()
    self.stage1_generator = Stage1Generator()
    self.stage1_discriminator = Stage1Discriminator()

  def train(self, X_train, train_embeddings, batch_size = 64, num_epochs = 600, z_dim = 100, c_dim = 128, stage1_generator_lr = 0.0004, stage1_discriminator_lr = 0.0004):
    generator_optimizer = tf.keras.optimizers.Adam(lr = stage1_generator_lr, beta_1 = 0.5, beta_2 = 0.999)
    discriminator_optimizer = tf.keras.optimizers.Adam(lr = stage1_discriminator_lr, beta_1 = 0.5, beta_2 = 0.999)
    
    for epoch in range(num_epochs):
      start_time = time.time()
      if epoch % 50 == 0:
        K.set_value(generator_optimizer.learning_rate, generator_optimizer.learning_rate / 2)
        K.set_value(discriminator_optimizer.learning_rate, discriminator_optimizer.learning_rate / 2)
    
      generator_loss_log = []
      discriminator_loss_log = []
      steps_per_epoch = X_train.shape[0] // batch_size
      for i in range(steps_per_epoch):
        image_batch = X_train[i*batch_size:(i + 1)*batch_size]
        image_batch = (image_batch - 127.5) / 127.5
        embedding_batch = train_embeddings[i*batch_size:(i + 1)*batch_size]
        z_noise = tf.random.normal((batch_size, z_dim))

        mismatched_images = tf.roll(image_batch, shift = 1, axis = 0)

        real_labels = tf.random.uniform(shape = (batch_size, ), minval = 0.9, maxval = 1.0)
        fake_labels = tf.random.uniform(shape = (batch_size, ), minval = 0.0, maxval = 0.1)
        mismatched_labels = tf.random.uniform(shape = (batch_size, ), minval = 0.0, maxval = 0.1)

        with tf.GradientTape() as generator_tape, tf.GradientTape() as discriminator_tape:
          fake_images, phi = self.stage1_generator([embedding_batch, z_noise])
          
          real_logits = self.stage1_discriminator([image_batch, embedding_batch])
          fake_logits = self.stage1_discriminator([fake_images, embedding_batch])
          mismatched_logits = self.stage1_discriminator([mismatched_images, embedding_batch])
          
          l_sup = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(real_labels, fake_logits))
          l_klreg = KL_loss(tf.random.normal((phi.shape[0], phi.shape[1])), phi)
          generator_loss = l_sup + 2.0*l_klreg
          
          l_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(real_labels, real_logits))
          l_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(fake_labels, fake_logits))
          l_mismatched = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(mismatched_labels, mismatched_logits))
          discriminator_loss = 0.5*tf.add(l_real, 0.5*tf.add(l_fake, l_mismatched))
        
        generator_gradients = generator_tape.gradient(generator_loss, self.stage1_generator.trainable_variables+self.stage1_discriminator.trainable_variables)
        discriminator_gradients = discriminator_tape.gradient(discriminator_loss, self.stage1_discriminator.trainable_variables)
        
        generator_optimizer.apply_gradients(zip(generator_gradients, self.stage1_generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.stage1_discriminator.trainable_variables))
        
        generator_loss_log.append(generator_loss)
        discriminator_loss_log.append(discriminator_loss)

      end_time = time.time()

      if epoch % 1 == 0:
        epoch_time = end_time - start_time
        template = "Epoch {:03d}: generator_loss: {:.4f} - discriminator_loss: {:.4f} - epoch_time: {:.2f} s"
        print(template.format(epoch + 1, tf.reduce_mean(generator_loss_log), tf.reduce_mean(discriminator_loss_log), epoch_time))

      if epoch % 5 == 0 or epoch == num_epochs - 1:
        save_path = "/content/drive/My Drive/Text to Image/lr_results/epoch_" + str(epoch + 1)
        if os.path.exists(save_path) == False:
          os.makedirs(save_path)
        temp_batch_size = 10
        temp_z_noise = tf.random.normal((temp_batch_size, z_dim))
        temp_embedding_batch = train_embeddings[0:temp_batch_size]
        fake_images, _ = self.stage1_generator([temp_embedding_batch, temp_z_noise])
        for i, image in enumerate(fake_images):
          image = 127.5*image + 127.5
          image = image.numpy().astype('uint8')
          cv2.imwrite(save_path + "/gen_%d.png"%(i), image)
        self.stage1_generator.save_weights("/content/drive/My Drive/Text to Image/model_weights/stage1_generator_" + str(epoch + 1) + ".h5")
        self.stage1_discriminator.save_weights("/content/drive/My Drive/Text to Image/model_weights/stage1_discriminator_" + str(epoch + 1) + ".h5")

In [ ]:
if os.path.exists('/content/drive/My Drive/Text to Image/utility_data/X_train_lr.npy'):
  X_train_lr = np.load('/content/drive/My Drive/Text to Image/utility_data/X_train_lr.npy')
  Y_train_lr = np.load('/content/drive/My Drive/Text to Image/utility_data/Y_train_lr.npy')
  X_test_lr = np.load('/content/drive/My Drive/Text to Image/utility_data/X_test_lr.npy')
  Y_test_lr = np.load('/content/drive/My Drive/Text to Image/utility_data/Y_test_lr.npy')
  train_embeddings = np.load('/content/drive/My Drive/Text to Image/utility_data/train_embeddings.npy')
  test_embeddings = np.load('/content/drive/My Drive/Text to Image/utility_data/test_embeddings.npy')
  X_train_lr = X_train_lr.astype('float32')
  X_test_lr = X_test_lr.astype('float32')

else: 
  data_dir = "/content/drive/My Drive/Text to Image/dataset/birds"
  train_dir = data_dir + "/train"
  test_dir = data_dir + "/test"
  train_embeddings_path = train_dir + "/char-CNN-RNN-embeddings.pickle"
  test_embeddings_path = test_dir + "/char-CNN-RNN-embeddings.pickle"
  train_filenames_path = train_dir + "/filenames.pickle"
  test_filenames_path = test_dir + "/filenames.pickle"
  train_class_info_path = train_dir + "/class_info.pickle"
  test_class_info_path = test_dir + "/class_info.pickle"
  cub_dataset_dir = "/content/drive/My Drive/Text to Image/dataset/CUB_200_2011/CUB_200_2011"
  bounding_boxes_path = cub_dataset_dir + "/bounding_boxes.txt"
  image_names_path = cub_dataset_dir + "/images.txt"

  train_loader = DataLoader(train_filenames_path, train_class_info_path, train_embeddings_path, bounding_boxes_path, image_names_path, image_size = (64, 64))  
  X_train_lr, Y_train_lr, train_embeddings = train_loader.load_dataset()
  np.save('/content/drive/My Drive/Text to Image/utility_data/X_train_lr', X_train_lr)
  np.save('/content/drive/My Drive/Text to Image/utility_data/Y_train_lr', Y_train_lr)
  np.save('/content/drive/My Drive/Text to Image/utility_data/train_embeddings', train_embeddings)

  test_loader = DataLoader(test_filenames_path, test_class_info_path, test_embeddings_path, bounding_boxes_path, image_names_path, image_size = (64, 64))  
  X_test_lr, Y_test_lr, test_embeddings = test_loader.load_dataset()
  np.save('/content/drive/My Drive/Text to Image/utility_data/X_test_lr', X_test_lr)
  np.save('/content/drive/My Drive/Text to Image/utility_data/Y_test_lr', Y_test_lr)
  np.save('/content/drive/My Drive/Text to Image/utility_data/test_embeddings', test_embeddings)

In [ ]:
if os.path.exists('/content/drive/My Drive/Text to Image/utility_data/X_train_hr.npy'):
  X_train_hr = np.load('/content/drive/My Drive/Text to Image/utility_data/X_train_hr.npy')
  Y_train_hr = np.load('/content/drive/My Drive/Text to Image/utility_data/Y_train_hr.npy')
  X_test_hr = np.load('/content/drive/My Drive/Text to Image/utility_data/X_test_hr.npy')
  Y_test_hr = np.load('/content/drive/My Drive/Text to Image/utility_data/Y_test_hr.npy')
  train_embeddings = np.load('/content/drive/My Drive/Text to Image/utility_data/train_embeddings.npy')
  test_embeddings = np.load('/content/drive/My Drive/Text to Image/utility_data/test_embeddings.npy')

else:
  data_dir = "/content/drive/My Drive/Text to Image/dataset/birds"
  train_dir = data_dir + "/train"
  test_dir = data_dir + "/test"
  train_embeddings_path = train_dir + "/char-CNN-RNN-embeddings.pickle"
  test_embeddings_path = test_dir + "/char-CNN-RNN-embeddings.pickle"
  train_filenames_path = train_dir + "/filenames.pickle"
  test_filenames_path = test_dir + "/filenames.pickle"
  train_class_info_path = train_dir + "/class_info.pickle"
  test_class_info_path = test_dir + "/class_info.pickle"
  cub_dataset_dir = "/content/drive/My Drive/Text to Image/dataset/CUB_200_2011/CUB_200_2011"
  bounding_boxes_path = cub_dataset_dir + "/bounding_boxes.txt"
  image_names_path = cub_dataset_dir + "/images.txt"

  train_loader = DataLoader(train_filenames_path, train_class_info_path, train_embeddings_path, bounding_boxes_path, image_names_path, image_size = (256, 256))  
  X_train_hr, Y_train_hr, train_embeddings = train_loader.load_dataset()
  np.save('/content/drive/My Drive/Text to Image/utility_data/X_train_hr', X_train_hr)
  np.save('/content/drive/My Drive/Text to Image/utility_data/Y_train_hr', Y_train_hr)
  np.save('/content/drive/My Drive/Text to Image/utility_data/train_embeddings', train_embeddings)

  test_loader = DataLoader(test_filenames_path, test_class_info_path, test_embeddings_path, bounding_boxes_path, image_names_path, image_size = (256, 256))  
  X_test_lr, Y_test_lr, test_embeddings = test_loader.load_dataset()
  np.save('/content/drive/My Drive/Text to Image/utility_data/X_test_lr', X_test_hr)
  np.save('/content/drive/My Drive/Text to Image/utility_data/Y_test_lr', Y_test_hr)
  np.save('/content/drive/My Drive/Text to Image/utility_data/test_embeddings', test_embeddings)